In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
import numpy as np
import string

In [ ]:
def remove_punctuation(s):
  return s.translate(str.maketrans('', '', string.punctuation))

In [ ]:
def get_robert_frost():
    word2idx = {'START': 0, 'END': 1}
    current_idx = 2
    sentences = []
    max_length = 0
    for line in open('drive/MyDrive/robert_frost.txt',encoding='utf-8'):
        line = line.strip()
        if line:
            tokens = remove_punctuation(line.lower()).split()
            sentence = []
            for t in tokens:
                if t not in word2idx:
                    word2idx[t] = current_idx
                    current_idx += 1
                idx = word2idx[t]
                sentence.append(idx)
            sentence.append(1) # 1 meaning END
            max_length = max(max_length, len(sentence))
            sentences.append(sentence)
    return sentences, word2idx, max_length


def sentence2lists(sentence):
  l=[]
  for i in range(len(sentence)):
    l.append(sentence[:i+1])
  return l

def padding(sentence, max_length):
  return [0 for i in range(max_length- len(sentence))]+sentence

def final_data(sentences, max_l):
  l= []
  for s in sentences:
    l += sentence2lists(s)
  for i in range(len(l)):
    l[i] = padding(l[i], max_l)
    
  return np.array(l)




In [ ]:
def generate(model, pi, word2idx,n):
        # convert word2idx -> idx2word
  idx2word = {v:k for k,v in word2idx.items()}
  V = len(pi)

  n_lines = 0

  X = [0 for i in range(max_length-2) ]+[np.random.choice(V, p=pi) ]
  print(idx2word[X[-1]], end=" ")

  while n_lines < n:
    xx= np.array(X).reshape(1,-1)
    probs = model.predict(xx).reshape(-1,)
    word_idx = np.random.choice(V, p=probs)
    X.append(word_idx)
    X= X[1:]
    if word_idx > 1:
      # it's a real word, not start/end token
      word = idx2word[word_idx]
      print(word, end=" ")
    elif word_idx == 1:
      # end token
      n_lines += 1
      print('')
      if n_lines < n:
        X =[ 0 for i in range(max_length-2) ]+[ np.random.choice(V, p=pi) ]# reset to start of line
        print(idx2word[X[-1]], end=" ")
def generate_poetry(model,n):

    # determine initial state distribution for starting sentences
  V = len(word2idx)
  pi = np.zeros(V)
  for sentence in sentences:
    pi[sentence[0]] += 1
  pi /= pi.sum()

  generate(model,pi, word2idx,n)

In [ ]:
sentences, word2idx, max_length = get_robert_frost()
l = final_data(sentences,max_length)
N = len(sentences)
D = 100
V = len(word2idx)
X = l[:,:-1]
Y = tf.keras.utils.to_categorical( l[:,-1], num_classes = V)
print(V)
print(Y[1].shape)

2199
(2199,)


In [ ]:
model = Sequential()
model.add(Embedding(V, D, input_length = max_length-1))
#model.add(tf.keras.layers.BatchNormalization())
model.add(LSTM(100))
model.add(Dense(V, activation = 'softmax'))

model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(0.01), metrics = ['accuracy'])

history = model.fit(X,Y,epochs = 100)

Epoch 1/100
380/380 [==============================] - 4s 7ms/step - loss: 5.9785 - accuracy: 0.1359
Epoch 2/100
380/380 [==============================] - 3s 7ms/step - loss: 5.2687 - accuracy: 0.1578
Epoch 3/100
380/380 [==============================] - 3s 7ms/step - loss: 4.7598 - accuracy: 0.1829
Epoch 4/100
380/380 [==============================] - 3s 7ms/step - loss: 4.3174 - accuracy: 0.2103
Epoch 5/100
380/380 [==============================] - 3s 7ms/step - loss: 3.8397 - accuracy: 0.2435
Epoch 6/100
380/380 [==============================] - 3s 7ms/step - loss: 3.4695 - accuracy: 0.2882
Epoch 7/100
380/380 [==============================] - 3s 7ms/step - loss: 3.1283 - accuracy: 0.3310
Epoch 8/100
380/380 [==============================] - 3s 7ms/step - loss: 2.8736 - accuracy: 0.3663
Epoch 9/100
380/380 [==============================] - 3s 7ms/step - loss: 2.6619 - accuracy: 0.4023
Epoch 10/100
380/380 [==============================] - 3s 7ms/step - loss: 2.5327 - accura

In [ ]:
generate_poetry(model,10)

with the brook going to rain 
that made him throw his bare legs out of the grange 
you want my creaking feet 
to the eye of i had halted more on the earth 
that overflowing like will leave it inward on her books 
its for fifty dollars for one langshang cock 
i turned and repented but coming back 
she hadnt found one leg and a crutch 
to carry again to you 
shes let me say the man was after 
